Download data from [CryptoWatch API](https://cryptowat.ch/docs/api) for educational purposes.

In [1]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd

/Users/santiagobasulto/.virtualenvs/ds/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/santiagobasulto/.virtualenvs/ds/lib/python3.8/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Markets

In [2]:
resp = requests.get('https://api.cryptowat.ch/markets')

In [3]:
resp.ok

True

In [4]:
resp.json()['result'][:2]

[{'id': 1,
  'exchange': 'bitfinex',
  'pair': 'btcusd',
  'active': True,
  'route': 'https://api.cryptowat.ch/markets/bitfinex/btcusd'},
 {'id': 2,
  'exchange': 'bitfinex',
  'pair': 'ltcusd',
  'active': True,
  'route': 'https://api.cryptowat.ch/markets/bitfinex/ltcusd'}]

In [5]:
markets = pd.DataFrame.from_records(resp.json()['result'], index='id')
markets.head()

,exchange,pair,active,route
id,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd
3,bitfinex,ltcbtc,True,https://api.cryptowat.ch/markets/bitfinex/ltcbtc
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd
5,bitfinex,ethbtc,True,https://api.cryptowat.ch/markets/bitfinex/ethbtc


### Exchanges

In [6]:
VALID_EXCHANGES = [x.strip() for x in """
bitfinex
coinbase-pro
bitstamp
kraken
cexio
okcoin
bitmex
mexbt
huobi
poloniex
bittrex
okex
hitbtc
""".split('\n') if x.strip()]
VALID_EXCHANGES

['bitfinex',
 'coinbase-pro',
 'bitstamp',
 'kraken',
 'cexio',
 'okcoin',
 'bitmex',
 'mexbt',
 'huobi',
 'poloniex',
 'bittrex',
 'okex',
 'hitbtc']

### Cryptos

In [7]:
VALID_CRYPTOS = ['BTC', 'LTC', 'ETH']

In [8]:
CRYPTOS_REGEX = '|'.join(["^{}usdt*$".format(c.lower()) for c in VALID_CRYPTOS])
CRYPTOS_REGEX

'^btcusdt*$|^ltcusdt*$|^ethusdt*$'

### Pairs to download

In [9]:
markets.head()

,exchange,pair,active,route
id,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd
3,bitfinex,ltcbtc,True,https://api.cryptowat.ch/markets/bitfinex/ltcbtc
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd
5,bitfinex,ethbtc,True,https://api.cryptowat.ch/markets/bitfinex/ethbtc


In [10]:
markets['Should Download?'] = markets['pair'].str.contains(CRYPTOS_REGEX) & markets['exchange'].isin(VALID_EXCHANGES)

In [11]:
markets['symbol'] = markets['pair'].str[:3].str.upper()

In [12]:
# markets['OHLC URL'] = markets.apply(lambda row: URL_TEMPLATE.format(exchange=row['exchange'], pair=row['pair']), axis=1)

In [13]:
to_download = markets.loc[markets['Should Download?']]

In [14]:
to_download.head()

,exchange,pair,active,route,Should Download?,symbol
id,,,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd,True,BTC
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd,True,LTC
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd,True,ETH
65,coinbase-pro,btcusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,BTC
68,coinbase-pro,ethusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,ETH


In [15]:
to_download.shape

(46, 6)

### OHLC

In [16]:
url = 'https://api.cryptowat.ch/markets/bitfinex/ltcusdt/ohlc'

In [29]:
PERIODS = '86400' # Daily

In [17]:
resp = requests.get(url, params={'periods': PERIODS})

In [18]:
resp.ok

True

In [19]:
resp.json()['result']['86400'][:2]

[[1555113600, 77.3, 79.139, 77.3, 78.784, 4.51065, 353.00803185],
 [1555200000, 79.657, 80.909, 76.88, 77.439, 224.80873319, 17956.80318777912]]

From:

In [22]:
pd.to_datetime(resp.json()['result']['86400'][0][0], unit='s')

Timestamp('2019-04-13 00:00:00')

Up to:

In [23]:
pd.to_datetime(resp.json()['result']['86400'][-1][0], unit='s')

Timestamp('2020-04-30 00:00:00')

In [30]:
ohlc = pd.DataFrame.from_records(
    resp.json()['result'][PERIODS],
    columns=['CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'VolumeUSD']
)

In [31]:
ohlc['DateTime'] = pd.to_datetime(ohlc['CloseTime'], unit='s')

In [32]:
ohlc.head()

,CloseTime,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,VolumeUSD,DateTime
0,1555113600,77.300,79.139,77.300,78.784,4.510650,353.008032,2019-04-13
1,1555200000,79.657,80.909,76.880,77.439,224.808733,17956.803188,2019-04-14
2,1555286400,77.400,82.320,76.725,82.320,211.321604,16416.498960,2019-04-15
3,1555372800,81.183,81.505,74.500,78.892,227.123199,17550.141392,2019-04-16
4,1555459200,78.204,80.680,78.124,80.367,80.081169,6280.867545,2019-04-17


In [33]:
ohlc.shape

(384, 8)

### Download all pairs

In [34]:
to_download.head()

,exchange,pair,active,route,Should Download?,symbol
id,,,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd,True,BTC
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd,True,LTC
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd,True,ETH
65,coinbase-pro,btcusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,BTC
68,coinbase-pro,ethusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,ETH


In [37]:
URL_TEMPLATE = 'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc'
URL_TEMPLATE

'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc'

In [57]:
def download_ohlc_into_csv(exchange, pair, symbol, destination='crypto_data'):
    url = URL_TEMPLATE.format(exchange=exchange, pair=pair)
    
    identifier = '{exchange}_{symbol}'.format(exchange=exchange, symbol=symbol.lower())

    resp = requests.get(url, params={'periods': 86400})
    if not resp.ok:
        return (identifier, False, 'Invalid response')
    
    if not resp.text:
        return (identifier, False, 'Empty Data')
    try:
        data = resp.json()['result']['86400']
    except json.JSONDecodeError:
        return (identifier, False, 'Invalid JSON')

    df = pd.DataFrame.from_records(
        data,
        columns=['CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'VolumeUSD']
    )

    df['DateTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    file_name = "{}/{}.csv".format(destination, identifier)
    df.to_csv(file_name, index=False)
    return (identifier, True, '')

Just a quick test:

In [48]:
download_ohlc_into_csv('coinbase-pro', 'ethusd', 'ETH')

('coinbase-pro_eth', True, '')

In [49]:
!head -n 3 crypto_data/coinbase-pro_eth.csv

CloseTime,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,VolumeUSD,DateTime
1463702400,13.18,14.9,13.0,14.9,950.441205,0.0,2016-05-20
1463788800,14.9,14.82,13.71,14.17,254.166617,0.0,2016-05-21


In [51]:
with ThreadPoolExecutor(max_workers=10) as ex:
    for exchange, pair, symbol in to_download[['exchange', 'pair', 'symbol']].values:
        futures = [ex.submit(download_ohlc_into_csv, exchange, pair, symbol)]
        for future in as_completed(futures):
            print(future.result())

('bitfinex_btc', True, '')
('bitfinex_ltc', True, '')
('bitfinex_eth', True, '')
('coinbase-pro_btc', True, '')
('coinbase-pro_eth', True, '')
('coinbase-pro_ltc', True, '')
('bitstamp_btc', True, '')
('bitstamp_ltc', True, '')
('bitstamp_eth', True, '')
('kraken_btc', True, '')
('kraken_ltc', True, '')
('kraken_eth', True, '')
('cexio_btc', True, '')
('cexio_ltc', True, '')
('cexio_eth', True, '')
('okcoin_btc', True, '')
('okcoin_ltc', False, 'Invalid JSON')
('mexbt_btc', True, '')
('poloniex_eth', True, '')
('poloniex_btc', True, '')
('poloniex_ltc', True, '')
('bittrex_btc', True, '')
('bittrex_eth', True, '')
('bittrex_ltc', True, '')
('okex_btc', True, '')
('okex_ltc', True, '')
('okex_eth', True, '')
('hitbtc_ltc', True, '')
('hitbtc_btc', True, '')
('bittrex_btc', True, '')
('hitbtc_eth', True, '')
('bittrex_eth', True, '')
('huobi_btc', True, '')
('huobi_ltc', True, '')
('huobi_eth', True, '')
('bittrex_ltc', True, '')
('bitfinex_btc', True, '')
('bitfinex_eth', True, '')
('bi